<a href="https://colab.research.google.com/github/alexcpass/python_ad/blob/main/Analise_vendas_3tabelas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Trabalho de analise de dados a partir de 3 tabelas:vendas; produtos; cadastro

In [ ]:
#importando bibliotecas
import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sns

In [ ]:
cadastro = pd.read_csv('/content/cadastro.csv',sep=',')
produto = pd.read_csv('/content/produto.csv',sep=',')
vendas = pd.read_csv('/content/vendas.csv',sep=',')

In [ ]:
cadastro.head(2)

In [ ]:
produto.head(2)

In [ ]:
vendas.head(2)

Vamos conectar as tabelas atraves dos campos em comum:
*   vendas e cadastro = id_cliente
*   vendas e produto = id_produto



In [ ]:
# Juntando tabelas
df = pd.merge(vendas, cadastro, on='id_cliente')
df = pd.merge(df, produto, on='id_produto')


In [ ]:
df.head()

In [ ]:
df.shape

Tratando os campos de data

In [ ]:
from datetime import datetime

In [ ]:
df['data_venda'] = pd.to_datetime(df['data_venda'])


In [ ]:
df.head(2)

Analises estatisticas/descritivas

In [ ]:
df.describe()

Correlação entre Variáveis:

In [ ]:
correlation_matrix = df[['quantidade', 'preco_produto']].corr()
print(correlation_matrix)


Análise de Vendas por Produto:

In [ ]:
total_vendas_por_produto = df.groupby('nome_produto')['quantidade'].sum().reset_index()
total_vendas_por_produto

Análise Temporal de Vendas:

In [ ]:
vendas_por_mes = df.groupby(df['data_venda'].dt.month)['quantidade'].sum().reset_index()
vendas_por_mes

Análise de Clientes:

In [ ]:
compras_por_cliente = df.groupby('nome_cliente')['quantidade'].mean().reset_index()
compras_por_cliente.round(2)

Comparação de Vendas entre Estados ou Cidades:

In [ ]:
vendas_por_estado = df.groupby('estado')['quantidade'].sum().reset_index()
vendas_por_estado

##Grafico de barras - Vendas por Estado (em qtd)

In [ ]:
ax = sns.barplot(vendas_por_estado, x="estado", y="quantidade", estimator="sum", errorbar=None)
ax.bar_label(ax.containers[0], fontsize=10);

##Grafico de barras - Vendas por cliente (em qtd)

In [ ]:
ax = sns.barplot(compras_por_cliente.round(), x="quantidade", y="nome_cliente", estimator="sum", errorbar=None, orient='h')
ax.bar_label(ax.containers[0], fontsize=10);

#Vamos tratar agora o valor das vendas, ao longo dos anos

Em primeiro lugar, vamos criar a coluna valor_venda, a partir da coluna quantidade x preco_produto

In [ ]:
# Criando a coluna valor_venda
df['valor_venda'] = df['quantidade'] * df['preco_produto']

# Exibindo o DataFrame atualizado
df.head()


In [ ]:
df['data_venda'] = pd.to_datetime(df['data_venda'])

# Adicione colunas de mês e ano
df['ano'] = df['data_venda'].dt.year
df['mes'] = df['data_venda'].dt.month

# Agrupe os dados por ano e mês e calcule o valor total de vendas para cada grupo
vendas_por_ano_mes = df.groupby(['ano', 'mes'])['valor_venda'].sum().reset_index()

# Visualize o resultado
vendas_por_ano_mes.round()


In [ ]:
plt.figure(figsize=(12, 6))
for ano in vendas_por_ano_mes['ano'].unique():
    dados_ano = vendas_por_ano_mes[vendas_por_ano_mes['ano'] == ano]
    plt.plot(dados_ano['mes'], dados_ano['valor_venda'], label=f'Ano {ano}')

plt.title('Vendas por Mês, Ano a Ano')
plt.xlabel('Mês')
plt.ylabel('Valor de Vendas')
plt.legend()
plt.show()


In [ ]:
pivot_table = vendas_por_ano_mes.pivot_table(values='valor_venda', index='mes', columns='ano', aggfunc='sum')

# Criar um gráfico de área empilhada
plt.figure(figsize=(12, 6))
sns.lineplot(data=pivot_table, dashes=False, palette='tab10')

plt.title('Vendas por Mês, Ano a Ano (Área Empilhada)')
plt.xlabel('Mês')
plt.ylabel('Valor de Vendas')
plt.legend(title='Ano', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.show()


In [ ]:
pivot_table = vendas_por_ano_mes.pivot_table(values='valor_venda', index='mes', columns='ano', aggfunc='sum')

# Criar um gráfico de área empilhada com linhas suaves
plt.figure(figsize=(12, 6))
sns.lineplot(data=pivot_table, dashes=False, palette='tab10', estimator='sum', ci=None)

plt.title('Vendas por Mês, Ano a Ano (Área Empilhada com Linhas Suaves)')
plt.xlabel('Mês')
plt.ylabel('Valor de Vendas')
plt.legend(title='Ano', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.show()


#Grafico de calor (valor das vendas)

In [ ]:
# Agrupe os dados por ano e mês e calcule o valor total de vendas para cada grupo
vendas_por_ano_mes = df.groupby(['ano', 'mes'])['valor_venda'].sum().reset_index()

# Crie uma tabela pivô para organizar os dados para o heatmap
pivot_table = vendas_por_ano_mes.pivot_table(values='valor_venda', index='mes', columns='ano', aggfunc='sum')

# Encontre os maiores valores para destacar
max_values = pivot_table.max(axis=1).max()

# Crie o heatmap usando Seaborn e destaque os maiores valores
plt.figure(figsize=(12, 8))
sns.heatmap(pivot_table, annot=True, fmt='.0f', cmap='YlGnBu', linewidths=.5, annot_kws={'weight': 'bold'},
            mask=(pivot_table == max_values))
plt.title('Vendas Mensais Ano a Ano (heatmap com destaque para os maiores valores)')
plt.xlabel('Ano')
plt.ylabel('Mês')
plt.show()


#Grafico de calor - valor + qtd

In [ ]:
# Agrupe os dados por ano, mês e calcule o valor total de vendas e a quantidade para cada grupo
vendas_por_ano_mes = df.groupby(['ano', 'mes'])[['valor_venda', 'quantidade']].sum().reset_index()

# Crie uma tabela pivô para organizar os dados para o heatmap
pivot_table = vendas_por_ano_mes.pivot_table(values=['valor_venda', 'quantidade'], index='mes', columns='ano', aggfunc='sum')

# Encontre os maiores valores para destacar
max_values = pivot_table.max(axis=1).max()

# Crie o heatmap usando Seaborn e destaque os maiores valores
plt.figure(figsize=(12, 8))
sns.heatmap(pivot_table, annot=True, fmt='.0f', cmap='YlGnBu', linewidths=.5, annot_kws={'weight': 'bold'},
            mask=(pivot_table == max_values))
plt.title('Vendas Mensais Ano a Ano (heatmap com destaque para os maiores valores)')
plt.xlabel('Ano')
plt.ylabel('Mês')
plt.show()


#Top 5 Clientes - qtd

In [ ]:
# Agrupe os dados por ano, cliente e calcule a quantidade total de vendas para cada grupo
vendas_por_ano_cliente = df.groupby(['ano', 'nome_cliente'])['quantidade'].sum().reset_index()

# Encontre os top 5 clientes por ano
top5_clientes_por_ano = vendas_por_ano_cliente.groupby('ano').apply(lambda x: x.nlargest(5, 'quantidade')).reset_index(drop=True)

# Crie um gráfico de barras empilhadas
plt.figure(figsize=(12, 8))

for ano, dados in top5_clientes_por_ano.groupby('ano'):
    bars = plt.bar(dados['nome_cliente'], dados['quantidade'], label=str(ano))

    # Adicione rótulos nas barras
    for bar in bars:
        yval = bar.get_height()
        plt.text(bar.get_x() + bar.get_width()/2, yval, round(yval, 1), ha='center', va='bottom', fontsize=10)

plt.title('Top 5 Clientes em Quantidade de Vendas por Ano')
plt.xlabel('Nome do Cliente')
plt.ylabel('Quantidade de Vendas')
plt.xticks(rotation=45, ha='right')
plt.legend(title='Ano', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()
